<a href="https://colab.research.google.com/github/adheeb4/7PAM2015-0105-2023_LLM/blob/main/LLM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate -U
!pip install datasets
!pip install rouge_score
!pip install evaluate
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "google-bert/bert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
num_parameters = model.num_parameters() / 1_000_000
print(f"Number of parameters: {round(num_parameters)}M")

Number of parameters: 110M


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
text = "You are [MASK]."

In [ ]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"{text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

You are beautiful.
You are right.
You are alive.
You are mine.
You are safe.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [ ]:
sample = dataset["train"].shuffle(seed=42).select(range(5))

for row in sample:
    print(f"\n'{row['text']}'")



'I stalk this truck.  I've been to industrial parks where I pretend to be a tech worker standing in line, strip mall parking lots, and of course the farmer's market.  The bowls are so so absolutely divine.  The owner is super friendly and he makes each bowl by hand with an incredible amount of pride.  You gotta eat here guys!!!'

'who really knows if this is good pho or not, i was hung tha fuck over and in desperate need of pho therapy. :P but it totally hit the spot and came out super freakin fast!!! omg! aaahhhhh.....\n\ni'm pretty sure it wasn't bad pho tho...meat, noodles, broth, all a-ok. the coffee was good too. thought i was gettin ripped off for a $3 cup of coffee but they gave me a big cup so it's all good! :)\n\nima make pho a must the next time i go to vegas again fo sure!!! yum! :D'

'I LOVE Bloom Salon... all of their stylist are very qualified and provide excellent hair care...I prefer to book my appointments with Andrea, but if she is not available I am not afraid to bo

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1019 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [ ]:
tokenizer.model_max_length

512

In [ ]:
chunk_size = 128

In [ ]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'Text {idx} length: {len(sample)}'")

'Text 0 length: 122'
'Text 1 length: 150'
'Text 2 length: 120'


In [ ]:
print(tokenized_samples)

{'input_ids': [[101, 2852, 1012, 18522, 4107, 2673, 1045, 2298, 2005, 1999, 1037, 2236, 18742, 1012, 2002, 1005, 1055, 3835, 1998, 3733, 2000, 2831, 2000, 2302, 2108, 9161, 6026, 1025, 2002, 1005, 1055, 2467, 2006, 2051, 1999, 3773, 2010, 5022, 1025, 2002, 1005, 1055, 6989, 2007, 1037, 2327, 1011, 18624, 2902, 1006, 27935, 1007, 2029, 2026, 3008, 2031, 4541, 2000, 2033, 2003, 2200, 2590, 1999, 2553, 2242, 6433, 1998, 2017, 2342, 5970, 1025, 1998, 2017, 2064, 2131, 6523, 7941, 2015, 2000, 2156, 15744, 2302, 2383, 2000, 2156, 2032, 2034, 1012, 2428, 1010, 2054, 2062, 2079, 2017, 2342, 1029, 1045, 1005, 1049, 3564, 2182, 2667, 2000, 2228, 1997, 2151, 10821, 1045, 2031, 2055, 2032, 1010, 2021, 1045, 1005, 1049, 2428, 5059, 1037, 8744, 1012, 102], [101, 6854, 1010, 1996, 9135, 1997, 2108, 2852, 1012, 18522, 1005, 1055, 5776, 2003, 1037, 9377, 1997, 1996, 3325, 1045, 1005, 2310, 2018, 2007, 2061, 2116, 2060, 7435, 1999, 16392, 1011, 1011, 2204, 3460, 1010, 6659, 3095, 1012, 2009, 3849, 2008,

In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 392'


In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 8'


In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result


In [ ]:
lm_datasets = tokenized_datasets.map(lambda examples: group_texts(examples), batched=True)
lm_datasets

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 917280
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 70602
    })
})

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] dr [MASK] goldberg offers everything i look photographer [MASK] a general practitioner. he'[MASK] [MASK] [MASK] [MASK] [MASK] talk [MASK] without being patronizing ; he's always on [unused323] in [MASK] his patients ; he's affiliated with a top - notch hospital ( nyu ) which my [MASK] have explained to me [MASK] very important in case something [MASK] enormous you need surgery ; and you can [MASK] referrals to see specialists without having to [MASK] him first. really, what more do you need? i'm sitting here trying to [MASK] of any complaints i have about him, but i'm really drawing [MASK] [MASK]. [SEP] [CLS] unfortunately, the [MASK] of'

'>>> being dr. [MASK]'s patient is a repeat of the experience [MASK]'ve had with lopez many [MASK] doctors in nyc [MASK] - good doctor, terrible staff. it seems that his staff simply [MASK] answers the phone. it [MASK] takes anxiety hours of repeated calling to buff [MASK] [MASK]. [MASK] has time for that [MASK] [MASK] to deal with it? i 

In [ ]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] dr. goldberg offers everything i look for in a general practitioner. he's nice and easy to talk [MASK] without [MASK] patronizing [MASK] he's always on time in [MASK] his patients ; he [MASK] s [MASK] with a top [MASK] notch hospital ( nyu [MASK] which my parents have explained to me is very important in case something [MASK] and [MASK] need surgery ; and you [MASK] get [MASK] [MASK] [MASK] to see [MASK] without having to [MASK] him [MASK]. really [MASK] what more do [MASK] need [MASK] [MASK]'m sitting here [MASK] to think [MASK] any complaints i have [MASK] him [MASK] but i'm really drawing [MASK] blank. [SEP] [CLS] unfortunately, the frustration of'

'>>> [MASK] dr. goldberg'[MASK] patient [MASK] a repeat of the experience [MASK]'ve had with so many other doctors in [MASK] - - good doctor [MASK] terrible staff. [MASK] seems that his staff simply never answers the [MASK] [MASK] it usually [MASK] 2 hours of repeated calling to get an answer. [MASK] has time [MASK] that or [

In [ ]:
train_size = 100_000
test_size = int(0.1 * train_size)

subsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
subsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments

batch_size = 64
logging_steps = len(dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-yelp",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=subsampled_dataset["train"],
    eval_dataset=subsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 20.07


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.push_to_hub()

In [ ]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [ ]:
subsampled_dataset = subsampled_dataset.remove_columns(["word_ids"])
eval_dataset = subsampled_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=subsampled_dataset["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_token_type_ids": "token_type_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)


In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 64
train_dataloader = DataLoader(
    subsampled_dataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import get_full_repo_name

model_name = "google-bert-base-uncased-finetuned-yelp-accelerate"
repo_name = get_full_repo_name(model_name)
print(repo_name)

In [ ]:
from huggingface_hub import Repository

output_dir = model_name
repo = Repository(output_dir, clone_from=repo_name)

In [ ]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f"Epoch {epoch+1}: Perplexity: {perplexity}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch+1}", blocking=False
        )

In [ ]:
summary_model_checkpoint = "t5-small"

summary_tokenizer = AutoTokenizer.from_pretrained(summary_model_checkpoint)

In [ ]:
summary_dataset = load_dataset("cnn_dailymail", "3.0.0")

In [ ]:
print(summary_dataset)

In [ ]:

import re


# Function to clean the text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text) # Remove text inside parentheses
    text = re.sub('"','', text) # Remove double quotes
    text = re.sub("'",'', text) # Remove single quotes
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces
    return text

# Clean the dataset
def clean_dataset(example):
    example['article'] = clean_text(example['article'])
    example['highlights'] = clean_text(example['highlights'])
    return example

summary_dataset = summary_dataset.map(clean_dataset)

# Print the first example to verify
print(summary_dataset['train'][0])

In [ ]:
summary_train_size = 100_000
summary_test_size = int(0.1 * summary_train_size)

summary_subsampled_dataset = summary_dataset["train"].train_test_split(
    train_size=summary_train_size, test_size=summary_test_size, seed=56
)
summary_subsampled_dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(summary_model_checkpoint)

In [ ]:
inputs = tokenizer("I loved watching the movie Amen")
inputs

In [ ]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

In [ ]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["highlights"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_summary_dataset = summary_subsampled_dataset.map(preprocess_function, batched=True)


In [ ]:
print(tokenized_summary_dataset)

In [ ]:
import evaluate

rouge_score = evaluate.load("rouge")

In [ ]:
import nltk

nltk.download("punkt")

In [ ]:
from nltk.tokenize import sent_tokenize


def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:5])


print(three_sentence_summary(summary_dataset["train"][1]["article"]))

In [ ]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["article"]]
    return metric.compute(predictions=summaries, references=dataset["highlights"])

In [ ]:
import pandas as pd

score = evaluate_baseline(summary_dataset["validation"], rouge_score)
rouge_dict = {rn: round(score[rn] * 100, 2) for rn in rouge_names}
rouge_dict

In [ ]:
from transformers import T5ForConditionalGeneration
summary_model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 6
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-cnn",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
print(tokenized_summary_dataset["train"][:5])

In [ ]:
tokenized_summary_dataset = tokenized_summary_dataset.remove_columns(
    summary_dataset["train"].column_names
)

In [ ]:
features = [tokenized_summary_dataset["train"][i] for i in range(2)]
data_collator(features)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_summary_dataset["train"],
    eval_dataset=tokenized_summary_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
